In [11]:
from bs4 import BeautifulSoup
import requests
import re
from tqdm import tqdm
import scholarly
import pandas as pd
import numpy as np
import math
from urllib.parse import urljoin

import io
import requests
import PyPDF2
from PyPDF2 import PdfFileReader, utils
from bs4 import BeautifulSoup
import urllib.request as urllib2
from tqdm import tqdm

from io import StringIO
import subprocess

import os
from os import startfile
from psutil import process_iter
import time
from time import sleep

import matplotlib.pyplot as plt
import textract
import slate3k as slate

from pdfminer.layout import LAParams
from pdfminer.pdfinterp  import PDFResourceManager
from pdfminer.pdfinterp  import PDFPageInterpreter

from pdfminer.converter import PDFPageAggregator

from pdfminer.pdfpage import PDFPage
from pdfminer.layout import LTTextBoxHorizontal

In [12]:
def get_text(url):
    r = requests.get(url)
    f = io.BytesIO(r.content)
    document = f
    rsrcmgr = PDFResourceManager()
    laparams = LAParams()
    device = PDFPageAggregator(rsrcmgr, laparams=laparams)
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    content = ''
    for page in PDFPage.get_pages(document):
        interpreter.process_page(page)
        layout = device.get_result()
        for element in layout:
            if isinstance(element, LTTextBoxHorizontal):
                content = content + element.get_text().replace('\n', " ")
    return content

In [14]:
main_url = r'https://citeseerx.ist.psu.edu/search?q=cloud+computing&submit.x=0&submit.y=0&sort=rlv&t=doc'
page = BeautifulSoup(requests.get(main_url).text, 'html.parser')
article_urls = []
for _ in tqdm(range(35)):
    next_page_url = urljoin(main_url, page.find('div', id='pager').find('a').get('href'))
    for url in page.find_all('div', class_='result'):
        article_urls.append(urljoin(main_url, url.find('a').get('href')))
    page =  BeautifulSoup(requests.get(next_page_url).text, 'html.parser')

100%|██████████████████████████████████████████████████████████████████████████████████| 35/35 [01:30<00:00,  2.49s/it]


In [17]:
titles = []
authors = []
dates = []
abstracts = []
urls = []
keyWord = []

for i in tqdm(range(len(article_urls))):
    page = BeautifulSoup(requests.get(article_urls[i]).text, 'lxml')
    try:
        titles.append(page.find_all('meta')[4].get('content'))
    except:
        titles.append('nan')
    try:
        authors.append(page.find_all('meta')[5].get('content'))
    except:
        authors.append('nan')
    try:
        dates.append(page.find_all('meta')[6].get('content'))
    except:
        dates.append('nan')
    try:
        abstracts.append(page.find('div', id='abstract').find('p').get_text())
    except:
        abstracts.append('nan')
    try:
        urls.append('https://citeseerx.ist.psu.edu' + page.find('ul', id='clinks').find('a').get('href'))
    except:
        urls.append('nan')
    key_words = []
    keys = page.find('div', id = 'keywords').find_all('a')
    for k in keys:
        key_words.append(k.get_text())
    keyWord.append(key_words)
    
kek = pd.DataFrame(
{
    'titles': titles,
    'authors':authors,
    'dates': dates,
    'abstracts': abstracts,
    'urls': urls,
    'keys': keyWord
})


 19%|███████████████▌                                                                 | 67/350 [01:53<08:14,  1.75s/it]


 39%|██████████████████████████████▊                                                 | 135/350 [03:55<05:47,  1.62s/it]


 58%|██████████████████████████████████████████████▍                                 | 203/350 [05:56<04:04,  1.67s/it]


 77%|█████████████████████████████████████████████████████████████▉                  | 271/350 [07:49<02:18,  1.75s/it]


 97%|█████████████████████████████████████████████████████████████████████████████▍  | 339/350 [09:40<00:18,  1.64s/it]


100%|████████████████████████████████████████████████████████████████████████████████| 350/350 [09:57<00:00,  1.62s/it]

In [20]:
article_url = 'https://citeseerx.ist.psu.edu' +kek.urls.values

In [22]:
texts = []
for i in tqdm(article_url):
    try:
        texts.append(get_text(i))
    except:
        texts.append(np.nan)
kek['texts'] = texts


100%|███████████████████████████████████████████████████████████████████████████████| 350/350 [00:00<00:00, 595.23it/s]

In [23]:
kek.to_csv('kek.csv', index = False)